In [ ]:
# data의 text를 통해서 disaster관련 문장이면 target 1 아니면 0

# initial method

# text -> tokenized 

# using pre trained word embedding (Word2Vec)

# GRU (bidirectional)

# result

# input data : text

# output data : target(digit 0,1)

#  model : architecture (GRU) // params : (embedding,GRU의 params)

# cost : binary_cross_entropy

In [2]:
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from pandas import DataFrame as df
import nltk
from matplotlib import pyplot as plt

In [7]:
data=pd.read_csv('./Desktop/train.csv',header=0)

In [38]:
# 1. text data pre processing
# cleanse, tokenize, encoding, vectorize

# 1.1. cleanse
import re
# 이전엔 그냥 str의 replace 활용했는데 #는 못바꾸넹..
# 대문자 -> 소문자
# 특수문자 제거 (구두점, 쉼표 제외)
def cleanse(sentence):
    result=sentence.lower()
    result=re.sub('[^0-9a-z ,.]','',result)
    return result
data['post_text']=data['text'].apply(lambda i : cleanse(i))
print(data.post_text[:10])

0    our deeds are the reason of this earthquake ma...
1               forest fire near la ronge sask. canada
2    all residents asked to shelter in place are be...
3    13,000 people receive wildfires evacuation ord...
4    just got sent this photo from ruby alaska as s...
5    rockyfire update  california hwy. 20 closed in...
6    flood disaster heavy rain causes flash floodin...
7    im on top of the hill and i can see a fire in ...
8    theres an emergency evacuation happening now i...
9    im afraid that the tornado is coming to our ar...
Name: post_text, dtype: object


In [39]:
# # of data
len(data) #7613
# train_dev split
from sklearn.model_selection import train_test_split
train_data,val_data=train_test_split(data,test_size=613) # train data 7000, val data 613

In [64]:
# nltk로 tokenize
# 1.2. tokenize
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# train_text_tokenized=word_tokenize(train_data.post_text) #-> error 발생
Result=[]
odd=[]
for i in train_data.index:
    result=word_tokenize(train_data.post_text[i])
    Result.append(result)
print(Result[:10])
train_data[:10]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\UNIMAX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[['megadeth', 'week', 'symphony', 'of', 'destruction', 'httpt.coecd7hizja1'], ['confirmed', 'the', 'debris', 'from', 'mh370'], ['must', 'get', 'hot', 'in', 'burning', 'buildingsloses', 'followers', 'httpst.co8sp8xlbbdr'], ['stormbeard', 'steellord', 'i', 'seen', 'judas', 'priest', 'in', '2005', 'when', 'rob', 'came', 'back', 'scorpions', 'as', 'support', '.', 'fucking', 'annihilated', 'the', 'place', '.', 'astonishing', 'gig'], ['losdelsonido', 'obama', 'declares', 'disaster', 'for', 'typhoondevastated', 'saipan', 'obama', 'signs', 'disaster', 'declaration', 'for', 'northern', 'ma', '...', 'ivanberroa'], ['hey', 'there', 'lonely', 'girldid', 'you', 'have', 'to', 'tell', 'your', 'friendsabout', 'the', 'way', 'i', 'got', 'you', 'screaming', 'my', 'name'], ['56', 'will', 'destroy', 'reg', 'c', 'competitiveness', '.', 'the', 'entire', 'region', 'will', 'b', 'oversaturated', '.', 'yes', 'brockton', 'gets', '12m', 'and', 'regc', 'commonwealth', 'ppc', 'and'], ['devinjoslyn', 'thunder', 'and'

,id,keyword,location,text,target,post_text
2633,3776,destruction,"Manchester, UK",Megadeth Week - Symphony Of Destruction http:/...,0,megadeth week symphony of destruction httpt.c...
2167,3108,debris,NaN,Confirmed the debris from MH370 ??,1,confirmed the debris from mh370
1223,1762,buildings%20burning,"England, United Kingdom",Must get hot in burning buildings\n\n-loses fo...,1,must get hot in burning buildingsloses followe...
232,331,annihilated,"West Lancashire, UK.",@stormbeard @steel_lord I seen Judas Priest in...,0,stormbeard steellord i seen judas priest in 20...
2719,3905,devastated,REPUBLICA DOMINICANA,(#LosDelSonido) Obama Declares Disaster for Ty...,1,losdelsonido obama declares disaster for typho...
5972,8526,screaming,"Aveiro, Portugal",Hey there lonely girl\nDid you have to tell yo...,0,hey there lonely girldid you have to tell your...
2564,3679,destroy,New England,5/6 will destroy Reg C competitiveness. The ...,1,56 will destroy reg c competitiveness. the e...
6676,9567,thunder,NaN,@DevinJoslyn thunder and lighting ????,0,devinjoslyn thunder and lighting
1351,1952,burning%20buildings,"Oklahoma City, OK",Large fire burning several buildings causing e...,1,large fire burning several buildings causing e...
5584,7969,razed,NaN,The Latest: More Homes Razed by Northern Calif...,1,the latest more homes razed by northern califo...


In [60]:
odd

[]

In [46]:
# 1.3 word embedding (vectorize+encoding)
from gensim.models import Word2Vec
word2vec=Word2Vec()